In [3]:
import sys
import numpy as np 
sys.path.append('../../src/')

from mask_dataset import MaskedDataset
dataset = MaskedDataset(model_name="mbert", reload=False)

Load pre-saved file /home/xzhao/workspace/probing-mulitlingual/result/dataset-mbert/mlama-dataset.pkl for MaskedDataset instance


In [7]:
from wiki_evaluation import plot_stacted_bar_and_percentage_of_fk_matching
plot_stacted_bar_and_percentage_of_fk_matching(dataset, resource_type='tokenized_article')

TypeError: plot_stacted_bar_and_percentage_of_fk_matching() got an unexpected keyword argument 'article'

In [5]:
from wiki_evaluation import get_all_and_matched_uuid_lsts
from collections import defaultdict
import re, os, pickle
from tqdm import tqdm 

root = "/home/xzhao/workspace/probing-mulitlingual/datasets/2018_dump_wiki_cache/sub_obj_ggrep"
dump_root = "/disk/xzhao/datasets/wikipedia_2018_octnov/sub_obj_ggrep_processed_result_dump"
os.makedirs(dump_root, exist_ok=True)
def return_allthings(lang, tokenized2sub_uri, tokenized2obj_uri, rel2all_uuid, rel2matched_uuid):
    dump_fn = os.path.join(dump_root, f"{lang}.pkl")
    if os.path.exists(dump_fn):
        with open(dump_fn, 'rb') as fp:
            return pickle.load(fp)

    lang_root = os.path.join(root, lang)
    suburi2lineid = defaultdict(set)
    objuri2lineid = defaultdict(set)

    for subdir in os.listdir(lang_root):
        subdir = os.path.join(lang_root, subdir)
        for idx in range(10):
            obj_match_fn = os.path.join(subdir, f"object_match.txt.0{idx}")
            sub_match_fn = os.path.join(subdir, f"subject_match.txt.0{idx}")
    
            
            file_id = subdir.split("/")[-1] + f":{idx}"
            with open(sub_match_fn, 'r') as fp:
                for line in fp:
                    line_id, *words = re.match(r'^(\d+):(.*)$', line.strip()).groups()
                    for word in map(str.strip, words[0].split('\t')):
                        for sub_uri in tokenized2sub_uri[word]:
                            suburi2lineid[sub_uri].add(f"{file_id}:{line_id}")
            with open(obj_match_fn, 'r') as fp:
                for line in fp:
                    line_id, *words = re.match(r'^(\d+):(.*)$', line.strip()).groups()
                    for word in map(str.strip, words[0].split('\t')):
                        for obj_uri in tokenized2obj_uri[word]:
                            objuri2lineid[obj_uri].add(f"{file_id}:{line_id}")

    # Read sub
    uuid_info = dataset.get_uuid_info()
    all_sub_obj_pairs = []
    match_sub_obj_pairs = []
    for rel, uuids in rel2all_uuid.items():
        all_sub_obj_pairs.extend([(uuid_info[rel][uuid]['sub_uri'], uuid_info[rel][uuid]['obj_uri']) for uuid in uuids])        
    for rel, uuids in rel2matched_uuid.items():
        match_sub_obj_pairs.extend([(uuid_info[rel][uuid]['sub_uri'], uuid_info[rel][uuid]['obj_uri']) for uuid in uuids])
    with open(dump_fn, 'wb') as fp:
        pickle.dump((suburi2lineid, objuri2lineid, all_sub_obj_pairs, match_sub_obj_pairs), fp)
    return suburi2lineid, objuri2lineid, all_sub_obj_pairs, match_sub_obj_pairs


In [6]:
from concurrent.futures import ProcessPoolExecutor, as_completed
lang2rel2all_uuid, lang2rel2matched_uuid = get_all_and_matched_uuid_lsts(dataset=dataset, reload=False)

lang2tokenized2sub_uri = defaultdict(lambda: defaultdict(set))
for sub_uri, lang2info in dataset.get_sub_info().items():
    for lang, info in lang2info.items():
        lang2tokenized2sub_uri[lang][' '.join(info['sub_tokens'])].add(sub_uri)
lang2tokenized2obj_uri = defaultdict(lambda: defaultdict(set))
for obj_uri, lang2info in dataset.get_obj_info().items():
    for lang, info in lang2info.items():
        lang2tokenized2obj_uri[lang][' '.join(info['obj_tokens'])].add(obj_uri)

lang2matching_measurement = {}
with ProcessPoolExecutor(max_workers=53) as executor:
    futures = []
    for lang in tqdm(dataset.langs, desc="Generating resource for measuring factual knowledge existence in wiki and matches by ML-LMs"):
        # return_allthings(lang, lang2tokenized2sub_uri[lang], lang2tokenized2obj_uri[lang], lang2rel2all_uuid[lang], lang2rel2matched_uuid[lang])
        dump_fn = os.path.join(dump_root, f"{lang}.pkl")
        futures.append(executor.submit(return_allthings, lang, lang2tokenized2sub_uri[lang], lang2tokenized2obj_uri[lang], lang2rel2all_uuid[lang], lang2rel2matched_uuid[lang]))
        # suburi2lineid, objuri2lineid, all_sub_obj_pairs, match_sub_obj_pairs = return_allthings(lang, lang2tokenized2sub_uri[lang], lang2tokenized2obj_uri[lang], lang2rel2all_uuid[lang], lang2rel2matched_uuid[lang])    
    for ft in tqdm(as_completed(futures)):
        suburi2lineid, objuri2lineid, all_sub_obj_pairs, match_sub_obj_pairs = ft.result()
        # lang2matching_measurement[lang] = (suburi2lineid, objuri2lineid, all_sub_obj_pairs, match_sub_obj_pairs)

Generating resource for measuring factual knowledge existence in wiki and matches by ML-LMs: 100%|██████████| 53/53 [00:05<00:00,  9.57it/s]
0it [00:14, ?it/s]


KeyboardInterrupt: 

In [ ]:
from pred_evaluation import calculate_overall_p1_score_standard, get_full_match_matrix, calculate_overall_p1_score_from_match_matrix
import numpy as np 
langs, p1 = calculate_overall_p1_score_standard(dataset)

In [ ]:
from pred_evaluation import get_all_and_matched_uuid_lsts
from wiki_evaluation import _get_subject_object_cooccurence_in_article

def display_matches_and_wiki_info(dataset, tgt_lang, display_content=False):
    uuid2wikimatches = _get_subject_object_cooccurence_in_article(dataset, dataset.langs)[tgt_lang]
    rel2predicted_uuid = get_all_and_matched_uuid_lsts(dataset=dataset, reload=False)[1][tgt_lang]
    predicted_uuids = []
    for rel in rel2predicted_uuid:
        predicted_uuids.extend(rel2predicted_uuid[rel])
    
    nonwiki_uuids = [uuid for uuid in predicted_uuids if uuid2wikimatches[uuid] == 0]
    print(f"Matched factual knowledge in {tgt_lang}: {len(predicted_uuids)}")
    print(f"Matched factual knowledge & No occurrence in wiki in {tgt_lang}: {len(nonwiki_uuids)}")
    print(f"Non-wiki & Predicted Rate: {len(nonwiki_uuids)/len(predicted_uuids)}")
    
    for rel in rel2predicted_uuid:
        if len(rel2predicted_uuid[rel]) > 0:
            nonwiki_uuids_by_rel = [uuid for uuid in rel2predicted_uuid[rel] if uuid in nonwiki_uuids]
            print(f"{dataset.display_rel(rel)}\nall-matches: {len(rel2predicted_uuid[rel])}, nonwiki-matches: {len(nonwiki_uuids_by_rel)}")
            if nonwiki_uuids_by_rel:
                print(f"{dataset.display_uuid(nonwiki_uuids_by_rel[-1])}")
                print(f"{dataset.display_uuid(nonwiki_uuids_by_rel[-1], lang=tgt_lang)}\n")
            else:
                print(f"{dataset.display_uuid(rel2predicted_uuid[rel][-1])}")
                print(f"{dataset.display_uuid(rel2predicted_uuid[rel][-1], lang=tgt_lang)}\n")

display_matches_and_wiki_info(dataset, "af")

In [ ]:
# from wiki_evaluation import get_wiki_matches_matrix_from_dumped_wiki_article, _get_subject_object_cooccurence_in_article
# lang2uuid2matches = _get_subject_object_cooccurence_in_article(dataset, candidate_langs=dataset.langs, reload=True)
# langs, all_uuids, sub_matrix = get_wiki_matches_matrix_from_dumped_wiki_article(dataset, reload=True)

from wiki_evaluation import plot_stacted_bar_and_percentage_of_fk_matching
plot_stacted_bar_and_percentage_of_fk_matching(dataset, resource_type="article")

In [ ]:
len(np.where(sub_matrix[0]==2)[0])

In [ ]:
for uuid in lang2uuid2matches['zh']:
    if lang2uuid2matches['zh'][uuid]


In [ ]:
nonwiki_uuid_idxs = np.where(sub_matrix[langs.index('zh')]==2)[0]
uuid_infos = dataset.get_uuid_info_all_lang()
for idx in nonwiki_uuid_idxs[:30]:
    uuid_info = uuid_infos[all_uuids[idx]]['zh']
    print(uuid_info['sub'], uuid_info['obj'])


In [ ]:
data_path = "/home/xzhao/workspace/probing-mulitlingual/result/evaluation-mbert/wiki_and_probing_matching_matrix_from_dumped_wiki_article.pkl"
import pickle 
with open(data_path, 'rb') as fp:
    langs, all_uuids, sub_matrix = pickle.load(fp)

# sub_matrix[langs.index('en')].shape
len(np.where(sub_matrix[langs.index('zh')]==2)[0])

In [ ]:
import os
from wiki_evaluation import get_uri2file2lineids_from_article_grep_matching

TGT_DATA_ROOT = "/disk/xzhao/probing-multilingual/2018_dump_wiki_cache"
lang = 'en'
sub_grep_matching_root = os.path.join(TGT_DATA_ROOT, "article_grep_matched", 'sub', lang)
obj_grep_matching_root = os.path.join(TGT_DATA_ROOT, "article_grep_matched", 'obj', lang)

sub2file2ids = get_uri2file2lineids_from_article_grep_matching(sub_grep_matching_root, 'subject')
obj2file2ids = get_uri2file2lineids_from_article_grep_matching(obj_grep_matching_root, 'object')


In [ ]:
for uuid in np.where(sub_matrix[0]==2)[0][:10]:
    print(all_uuids[uuid])

In [ ]:
from wiki_evaluation import _get_subject_object_cooccurence_in_article, get_all_and_matched_uuid_lsts, measure_crosslingual_transfer_by_correlation_corrtpred_inwiki_fk
sorted_langs, lang2rate = measure_crosslingual_transfer_by_correlation_corrtpred_inwiki_fk(dataset, per_lang=True)

In [ ]:
import matplotlib.pyplot as plt

def draw_single(x_langs, line_langs, lang2rate):
    fig, ax = plt.subplots(figsize=(8,6))
    x = np.array(list(range(len(x_langs))))
    for lang in line_langs:
        ax.plot(x, np.array(lang2rate[lang]))
    ax.set_xticks(x)
    ax.set_xticklabels(x_langs, fontsize=8, rotation=90)
    plt.title("P1 scores for different relations per language", fontsize=12)
    ax.legend([dataset.display_lang(lang) for lang in x_langs], ncol=1, bbox_to_anchor=(1.0, 0.5), loc='center left')
    plt.show()

def draw_heatmap_for_pairwise_langsim_based_on_uuid_matching_rate(langs, lang2rate):
    transfer_matrix = np.zeros((len(langs), len(langs)))
    for base_lang in langs:
        for test_lang in langs:
            transfer_matrix[langs.index(base_lang)][langs.index(test_lang)] = lang2rate[test_lang][langs.index(base_lang)]
    
    import matplotlib.pyplot as plt
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(12,8))
    langs_name = [dataset.display_lang(lang) for lang in langs]
    g = sns.heatmap(transfer_matrix, xticklabels=langs_name, yticklabels=langs_name, cmap="GnBu") # type: ignore
    g.set_facecolor('#902008')
    plt.title("Possiblity of transfer knowledge from language in Y-axis to X-axis. \nThe languages are ranked by p1 scores", fontsize=12)

# draw_single(sorted_langs, sorted_langs[:10], lang2rate)
# draw_single(sorted_langs, ['af'], lang2rate)
draw_heatmap_for_pairwise_langsim_based_on_uuid_matching_rate(sorted_langs, lang2rate)

In [ ]:
from wiki_evaluation import plot_stacked_bar_and_percentage_of_inwiki_prediction
plot_stacked_bar_and_percentage_of_inwiki_prediction(dataset)

In [ ]:
from wiki_evaluation import _get_title_object_subject_matchings, plot_stacted_bar_and_percentage_of_fk_matching, get_wiki_matches_matrix_from_dumped_wiki_title
# aa = _get_title_object_subject_matchings(dataset=dataset, reload=True)
# get_wiki_matches_matrix_from_dumped_wiki_title(dataset=dataset, reload=True)
plot_stacted_bar_and_percentage_of_fk_matching(dataset, resource_type="title")

In [ ]:
from wiki_evaluation import get_wiki_matches_matrix_from_dumped_wiki_article, plot_stacted_bar_and_percentage_of_fk_matching, _get_subject_object_cooccurence_in_article
from wiki_2018_dump import _preprocess_uri2file2lineids_from_article_grep_matching
from wiki_evaluation import get_all_and_matched_uuid_lsts
# aa = _get_title_object_subject_matchings(dataset=dataset, reload=True)
# get_wiki_matches_matrix_from_dumped_wiki_title(dataset=dataset, reload=True)

# candidate_langs=['ca', 'da', 'fi', 'ga', 'he', 'ja', 'ka', 'ko', 'ms', 'nl', 'ru', 'sr', 'th', 'zh', 'it']
candidate_langs = dataset.langs
# _preprocess_uri2file2lineids_from_article_grep_matching(candidate_langs)
lang2uuid2wiki_matches = _get_subject_object_cooccurence_in_article(dataset, candidate_langs)
lang2rel2matched_uuid = get_all_and_matched_uuid_lsts(dataset=dataset, reload=False)[1]
plot_stacted_bar_and_percentage_of_fk_matching(dataset, resource_type="article", candidate_langs=candidate_langs)

In [ ]:
from wiki_evaluation import get_wiki_matches_matrix_from_dumped_wiki_article
langs, all_uuids, sub_matrix = get_wiki_matches_matrix_from_dumped_wiki_article(dataset)

In [ ]:
from wiki_evaluation import display_niwp_fk_by_article_matching
display_niwp_fk_by_article_matching(dataset, 'de')